In [1]:
import numpy as np
import pandas as pd

import os
import sys
from tqdm import tqdm_notebook as tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import PIL

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [3]:
#!wget https://storage.googleapis.com/openimages/challenge_2019/challenge-2019-classes-description-500.csv

In [4]:
class_lookup_df = pd.read_csv("./challenge-2019-classes-description-500.csv", header=None)
empty_submission_df = pd.read_csv("../input/sample_submission.csv")
empty_submission_df["PredictionString"] = [""] * len(empty_submission_df)
empty_submission_df.head()

,ImageId,PredictionString
0,b5d912e06f74e948,
1,be137cf6bb0b62d5,
2,8d65ca08cb5ce8e8,
3,4d3ad1e52ad8c065,
4,9b94408691c7d7bf,


In [5]:
class_lookup_df.columns = ["encoded_label","label"]
class_lookup_df['label'] = class_lookup_df['label'].str.lower()
class_lookup_df.head()

,encoded_label,label
0,/m/061hd_,infant bed
1,/m/06m11,rose
2,/m/03120,flag
3,/m/01kb5b,flashlight
4,/m/0120dh,sea turtle


In [6]:
ImageID_list = []
PredictionString_list = []

with tf.device('/device:GPU:0'):
    with tf.Graph().as_default():
        detector = hub.Module("https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1")
        image_string_placeholder = tf.placeholder(tf.string)
        decoded_image = tf.image.decode_jpeg(image_string_placeholder)
        # Module accepts as input tensors of shape [1, height, width, 3], i.e. batch
        # of size 1 and type tf.float32.
        decoded_image_float = tf.image.convert_image_dtype(
          image=decoded_image, dtype=tf.float32)
        module_input = tf.expand_dims(decoded_image_float, 0)
        result = detector(module_input, as_dict=True)
        init_ops = [tf.global_variables_initializer(), tf.tables_initializer()]

        session = tf.Session()
        session.run(init_ops)

        for file_id in tqdm(empty_submission_df["ImageId"].values, total=len(empty_submission_df)):
            with tf.gfile.Open("../input/test/"+file_id+".jpg", "rb") as binfile:
                image_string = binfile.read()

            result_out, image_out = session.run(
              [result, decoded_image],
              feed_dict={image_string_placeholder: image_string})
                        
            PredictionString = ""
            
            for box, score, labelname in zip(result_out["detection_boxes"],result_out["detection_scores"],result_out["detection_class_entities"]):
                labelname = labelname.decode('utf-8').lower()

                if class_lookup_df[class_lookup_df["label"] == labelname].shape[0] == 0:
                    # no match label
                    continue

                encoded_label = class_lookup_df[class_lookup_df["label"] == labelname]["encoded_label"].values[0]

                PredictionString += encoded_label 
                PredictionString += " "
                PredictionString += str(score)
                PredictionString += " "
                PredictionString += str(box[0])
                PredictionString += " "
                PredictionString += str(box[1])
                PredictionString += " "
                PredictionString += str(box[2])
                PredictionString += " "
                PredictionString += str(box[3])
                PredictionString += " "
            
            ImageID_list.append(file_id)
            PredictionString_list.append(PredictionString)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
results=pd.DataFrame({"ImageId":ImageID_list,
                      "PredictionString":PredictionString_list
                     })

In [8]:
results.head()

,ImageId,PredictionString
0,b5d912e06f74e948,/m/04yx4 0.85136557 0.28740385 0.4338511 0.796...
1,be137cf6bb0b62d5,/m/0hg7b 0.7785667 0.25795442 0.02194678 0.508...
2,8d65ca08cb5ce8e8,/m/0nybt 0.61177284 0.48958296 0.2892169 0.868...
3,4d3ad1e52ad8c065,/m/0dzct 0.88811165 0.07232189 0.4327386 0.301...
4,9b94408691c7d7bf,/m/063rgb 0.6159055 0.0 0.0 0.8935057 0.678049...


In [9]:
results.shape

(99999, 2)

In [10]:
results.to_csv("submission006.csv", index=False)